In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.main = nn.Sequential(
            nn.Conv2d(1, 64, 4, stride=1, padding=1),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(64, 128, 4, stride=1, padding=1),
            nn.InstanceNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(128, 256, 4, stride=1, padding=1),
            nn.InstanceNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(256, 512, 4, stride=1, padding=1),
            nn.InstanceNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(512, 1, 4, stride=1, padding=1),
        )

    def forward(self, x):
        x = self.main(x)
        x = F.avg_pool2d(x, x.size()[2:])
        x = torch.flatten(x, 1)
        return x


class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            # Initial convolution block
            nn.ReflectionPad2d(3),
            nn.Conv2d(1, 64, 7, stride=1,),
            nn.InstanceNorm2d(64),
            nn.ReLU(inplace=True),

            # Downsampling
            nn.Conv2d(64, 128, 3, stride=1, padding=1),
            nn.InstanceNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 256, 3, stride=1, padding=1),
            nn.InstanceNorm2d(256),
            nn.ReLU(inplace=True),

            # Residual blocks
            ResidualBlock(256),
            ResidualBlock(256),
            ResidualBlock(256),
            ResidualBlock(256),
            ResidualBlock(256),
            ResidualBlock(256),
            ResidualBlock(256),
            ResidualBlock(256),
            ResidualBlock(256),

            # Upsampling
            nn.ConvTranspose2d(256, 128, 3, stride=1, padding=1),
            nn.InstanceNorm2d(128),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(128, 64, 3, stride=1, padding=1),
            nn.InstanceNorm2d(64),
            nn.ReLU(inplace=True),

            # Output layer
            nn.ReflectionPad2d(3),
            nn.Conv2d(64, 1, 7),
            nn.Tanh()
        )

    def forward(self, x):
        return self.main(x)


class ResidualBlock(nn.Module):
    def __init__(self, in_channels):
        super(ResidualBlock, self).__init__()

        self.res = nn.Sequential(nn.ReflectionPad2d(1),
                                 nn.Conv2d(in_channels, in_channels, 3, stride=1),
                                 nn.InstanceNorm2d(in_channels),
                                 nn.ReLU(inplace=True),
                                 nn.ReflectionPad2d(1),
                                 nn.Conv2d(in_channels, in_channels, 3, stride=1),
                                 nn.InstanceNorm2d(in_channels))

    def forward(self, x):
        return x + self.res(x)


In [2]:
import pandas as pd 
import os
import glob
import os
import random
import time
from threading import Thread

import cv2
import numpy as np
from PIL import Image
from torch.utils.data import Dataset

class ImageDataset(Dataset):
    def __init__(self, root, transform=None, unaligned=False, mode="train", A="No Finding", B="Pneumonia"):
        efficentMode = True # optional
        cwd = os.getcwd()
        cwd = cwd[0:cwd.rfind("/")]
        db_list = f"{cwd}/data/Data_toy.csv"
        self.diseases_list = set()
        self.disease_count = {} # optional
        self.A = A
        self.B = B
        df = pd.read_csv(db_list)
        # print(df.head(10))
        if not efficentMode:
            for i, row in df.iterrows():
                    if '|' in row[1]:
                        diseases = row[1].split('|')
                        for d in diseases:
                            self.diseases_list.add(d)
                            if d not in df:
                                df[d] = 0
                                df.loc[i, d] = 1
                            else:
                                df.loc[i, d] = 1
                    else:
                        d = str(row[1])
                        self.diseases_list.add(d)
                        self.disease_count[d] = self.disease_count.get(d, 0) + 1
                        if d not in df:
                            df[d] = 0
                            df.loc[i, d] = 1
                        else:
                            df.loc[i, d] = 1
        else:
            A_df = df[df["Finding Labels"] == "No Finding"]
            B_df = df[df["Finding Labels"].str.match(B)]
        df.drop(columns=['Finding Labels'], inplace=True) # Optional. Disable if computation takes too long time
        # print(diseases_list)




        # A list of image_indices for every disease Hernia_List = [00000003_001.png, 00000003_002.png, 00000003_003.png....]


        # After getting one-hot encoding for each diseases, let's create two lists of image indices A and B. For now A will be No-Finding
        # and B will be a disease Pneumonia 
        self.images = {}

        # print(images)
        if not efficentMode:
            for i, d in enumerate(self.diseases_list):
                # print(f"--------------------{d}------------------------")
                self.images[d] = (df[df[d] == 1]["Image Index"].to_string(index=False).split("\n"))

        else:
            self.images[A] = A_df["Image Index"].to_string(index=False).split("\n")
            self.images[B] = B_df["Image Index"].to_string(index=False).split("\n")
        # print(self.images[B])
        # print(self.images)
###

        self.transform = transform
        self.unaligned = unaligned

        # self.files_A = sorted(glob.glob(os.path.join(root, f"{mode}/A") + "/*.*"))  # 
        # self.files_B = sorted(glob.glob(os.path.join(root, f"{mode}/B") + "/*.*")) # images

        


    def __getitem__(self, index):
        cwd = os.getcwd()
        cwd = cwd[0:cwd.rfind("/")]
        
        A_path = f"{cwd}/data/images/"+self.images[self.A][index % len(self.images[self.A])]
        if not self.unaligned:
            B_path = f"{cwd}/data/images/"+self.images[self.B][random.randint(0, len(self.images[self.B]) - 1)]
        else:
            B_path = f"{cwd}/data/images/"+self.images[self.B][index % len(self.images[self.B])]
        
#         print(A_path)
#         print(B_path)       
        
        A = Image.open(A_path)
        B = Image.open(B_path)
        A = A.convert("L")
        B = B.convert("L")
        item_A = self.transform(A)
        item_B = self.transform(B)
        
#         print("Pillow", A.mode, A.size)
#         print("PIllow", B.mode, B.size)
        return {"A": item_A, "B": item_B}



    def __len__(self):
        return max(len(self.images[self.A]), len(self.images[self.B]))


In [3]:
class DecayLR:
    def __init__(self, epochs, offset, decay_epochs):
        epoch_flag = epochs - decay_epochs
        assert (epoch_flag > 0), "Decay must start before the training session ends!"
        self.epochs = epochs
        self.offset = offset
        self.decay_epochs = decay_epochs

    def step(self, epoch):
        return 1.0 - max(0, epoch + self.offset - self.decay_epochs) / (
                self.epochs - self.decay_epochs)


In [4]:
import random

import torch


class ReplayBuffer:
    def __init__(self, max_size=50):
        assert (max_size > 0), "Empty buffer or trying to create a black hole. Be careful."
        self.max_size = max_size
        self.data = []

    def push_and_pop(self, data):
        to_return = []
        for element in data.data:
            element = torch.unsqueeze(element, 0)
            if len(self.data) < self.max_size:
                self.data.append(element)
                to_return.append(element)
            else:
                if random.uniform(0, 1) > 0.5:
                    i = random.randint(0, self.max_size - 1)
                    to_return.append(self.data[i].clone())
                    self.data[i] = element
                else:
                    to_return.append(element)
        return torch.cat(to_return)


# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight, 0.0, 0.02)
    elif classname.find("BatchNorm") != -1:
        torch.nn.init.normal_(m.weight, 1.0, 0.02)
        torch.nn.init.zeros_(m.bias)


In [ ]:
import argparse
import itertools
import os

import random

import torch.backends.cudnn as cudnn
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.utils as vutils
from PIL import Image
from tqdm import tqdm
import gc

dataset_name = "CXR"
output_dir = "outputs"
img_size = 1024
batch_size = 1
epochs_total = 10
print_freq = 100
lr_rate = 0.0002
decay_epochs = 5


try:
    os.makedirs(output_dir)
except OSError:
    pass

try:
    os.makedirs("weights")
except OSError:
    pass

manualSeed = random.randint(1, 10000)
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

cudnn.benchmark = True


# Dataset
dataset = ImageDataset(root=os.path.join("", dataset_name),
                       transform=transforms.Compose([
                           transforms.Resize(512, Image.BICUBIC),
                           transforms.RandomCrop(512),
                           transforms.RandomHorizontalFlip(),
                           transforms.ToTensor(),
                           transforms.Normalize((0.5,), (0.5,))]),
                       unaligned=True, A="No Finding", B="Mass")

dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, pin_memory=True)

try:
    os.makedirs(os.path.join(output_dir, dataset_name, "A"))
    os.makedirs(os.path.join(output_dir, dataset_name, "B"))
except OSError:
    pass

try:
    os.makedirs(os.path.join("weights", dataset_name))
except OSError:
    pass

device = torch.device("cuda:0")

# create model
netG_A2B = Generator().to(device)
netG_B2A = Generator().to(device)
netD_A = Discriminator().to(device)
netD_B = Discriminator().to(device)


# netG_A2B = nn.DataParallel(netG_A2B).to(device)
# netG_B2A = nn.DataParallel(netG_B2A).to(device)

# netD_A = nn.DataParallel(netD_A).to(device)
# netD_B = nn.DataParallel(netD_B).to(device)


netG_A2B.apply(weights_init)
netG_B2A.apply(weights_init)
netD_A.apply(weights_init)
netD_B.apply(weights_init)

# if args.netG_A2B != "":
#     netG_A2B.load_state_dict(torch.load(args.netG_A2B))
# if args.netG_B2A != "":
#     netG_B2A.load_state_dict(torch.load(args.netG_B2A))
# if args.netD_A != "":
#     netD_A.load_state_dict(torch.load(args.netD_A))
# if args.netD_B != "":
#     netD_B.load_state_dict(torch.load(args.netD_B))

# define loss function (adversarial_loss) and optimizer
cycle_loss = torch.nn.L1Loss().to(device)
identity_loss = torch.nn.L1Loss().to(device)
adversarial_loss = torch.nn.MSELoss().to(device)

# Optimizers
optimizer_G = torch.optim.Adam(itertools.chain(netG_A2B.parameters(), netG_B2A.parameters()),
                               lr=lr_rate, betas=(0.5, 0.999))
optimizer_D_A = torch.optim.Adam(netD_A.parameters(), lr=lr_rate, betas=(0.5, 0.999))
optimizer_D_B = torch.optim.Adam(netD_B.parameters(), lr=lr_rate, betas=(0.5, 0.999))

lr_lambda = DecayLR(epochs_total, 0, decay_epochs).step
lr_scheduler_G = torch.optim.lr_scheduler.LambdaLR(optimizer_G, lr_lambda=lr_lambda)
lr_scheduler_D_A = torch.optim.lr_scheduler.LambdaLR(optimizer_D_A, lr_lambda=lr_lambda)
lr_scheduler_D_B = torch.optim.lr_scheduler.LambdaLR(optimizer_D_B, lr_lambda=lr_lambda)

g_losses = []
d_losses = []

identity_losses = []
gan_losses = []
cycle_losses = []

fake_A_buffer = ReplayBuffer()
fake_B_buffer = ReplayBuffer()

for epoch in range(0, epochs_total):
    progress_bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for i, data in progress_bar:
        # get batch size data
        real_image_A = data["A"].to(device)
        real_image_B = data["B"].to(device)
        batch_size = real_image_A.size(0)

        # real data label is 1, fake data label is 0.
        real_label = torch.full((batch_size, 1), 1, device=device, dtype=torch.float32)
        fake_label = torch.full((batch_size, 1), 0, device=device, dtype=torch.float32)

        ##############################################
        # (1) Update G network: Generators A2B and B2A
        ##############################################

        # Set G_A and G_B's gradients to zero
        optimizer_G.zero_grad()

        # Identity loss
        # G_B2A(A) should equal A if real A is fed
        identity_image_A = netG_B2A(real_image_A)
        loss_identity_A = identity_loss(identity_image_A, real_image_A) * 5.0
        # G_A2B(B) should equal B if real B is fed
        identity_image_B = netG_A2B(real_image_B)
        loss_identity_B = identity_loss(identity_image_B, real_image_B) * 5.0

        # GAN loss
        # GAN loss D_A(G_A(A))
        fake_image_A = netG_B2A(real_image_B)
        fake_output_A = netD_A(fake_image_A)
        loss_GAN_B2A = adversarial_loss(fake_output_A, real_label)
        # GAN loss D_B(G_B(B))
        fake_image_B = netG_A2B(real_image_A)
        fake_output_B = netD_B(fake_image_B)
        loss_GAN_A2B = adversarial_loss(fake_output_B, real_label)

        # Cycle loss
        recovered_image_A = netG_B2A(fake_image_B)
        loss_cycle_ABA = cycle_loss(recovered_image_A, real_image_A) * 10.0

        recovered_image_B = netG_A2B(fake_image_A)
        loss_cycle_BAB = cycle_loss(recovered_image_B, real_image_B) * 10.0

        # Combined loss and calculate gradients
        errG = loss_identity_A + loss_identity_B + loss_GAN_A2B + loss_GAN_B2A + loss_cycle_ABA + loss_cycle_BAB

        # Calculate gradients for G_A and G_B
        errG.backward()
        # Update G_A and G_B's weights
        optimizer_G.step()

        ##############################################
        # (2) Update D network: Discriminator A
        ##############################################

        # Set D_A gradients to zero
        optimizer_D_A.zero_grad()

        # Real A image loss
        real_output_A = netD_A(real_image_A)
        errD_real_A = adversarial_loss(real_output_A, real_label)

        # Fake A image loss
        fake_image_A = fake_A_buffer.push_and_pop(fake_image_A)
        fake_output_A = netD_A(fake_image_A.detach())
        errD_fake_A = adversarial_loss(fake_output_A, fake_label)

        # Combined loss and calculate gradients
        errD_A = (errD_real_A + errD_fake_A) / 2

        # Calculate gradients for D_A
        errD_A.backward()
        # Update D_A weights
        optimizer_D_A.step()

        ##############################################
        # (3) Update D network: Discriminator B
        ##############################################

        # Set D_B gradients to zero
        optimizer_D_B.zero_grad()

        # Real B image loss
        real_output_B = netD_B(real_image_B)
        errD_real_B = adversarial_loss(real_output_B, real_label)

        # Fake B image loss
        fake_image_B = fake_B_buffer.push_and_pop(fake_image_B)
        fake_output_B = netD_B(fake_image_B.detach())
        errD_fake_B = adversarial_loss(fake_output_B, fake_label)

        # Combined loss and calculate gradients
        errD_B = (errD_real_B + errD_fake_B) / 2

        # Calculate gradients for D_B
        errD_B.backward()
        # Update D_B weights
        optimizer_D_B.step()

        progress_bar.set_description(
            f"[{epoch}/{epochs_total - 1}][{i}/{len(dataloader) - 1}] "
            f"Loss_D: {(errD_A + errD_B).item():.4f} "
            f"Loss_G: {errG.item():.4f} "
            f"Loss_G_identity: {(loss_identity_A + loss_identity_B).item():.4f} "
            f"loss_G_GAN: {(loss_GAN_A2B + loss_GAN_B2A).item():.4f} "
            f"loss_G_cycle: {(loss_cycle_ABA + loss_cycle_BAB).item():.4f}")

        if i % print_freq == 0:
            vutils.save_image(real_image_A,
                              f"{output_dir}/{dataset_name}/A/real_samples.png",
                              normalize=True)
            vutils.save_image(real_image_B,
                              f"{output_dir}/{dataset_name}/B/real_samples.png",
                              normalize=True)

            fake_image_A = 0.5 * (netG_B2A(real_image_B).data + 1.0)
            fake_image_B = 0.5 * (netG_A2B(real_image_A).data + 1.0)

            vutils.save_image(fake_image_A.detach(),
                              f"{output_dir}/{dataset_name}/A/fake_samples_epoch_{epoch}.png",
                              normalize=True)
            vutils.save_image(fake_image_B.detach(),
                              f"{output_dir}/{dataset_name}/B/fake_samples_epoch_{epoch}.png",
                              normalize=True)

    # do check pointing
    torch.save(netG_A2B.state_dict(), f"weights/{dataset_name}/netG_A2B_epoch_{epoch}.pth")
    torch.save(netG_B2A.state_dict(), f"weights/{dataset_name}/netG_B2A_epoch_{epoch}.pth")
    torch.save(netD_A.state_dict(), f"weights/{dataset_name}/netD_A_epoch_{epoch}.pth")
    torch.save(netD_B.state_dict(), f"weights/{dataset_name}/netD_B_epoch_{epoch}.pth")

    # Update learning rates
    lr_scheduler_G.step()
    lr_scheduler_D_A.step()
    lr_scheduler_D_B.step()

# save last check pointing
torch.save(netG_A2B.state_dict(), f"weights/{dataset_name}/netG_A2B.pth")
torch.save(netG_B2A.state_dict(), f"weights/{dataset_name}/netG_B2A.pth")
torch.save(netD_A.state_dict(), f"weights/{dataset_name}/netD_A.pth")
torch.save(netD_B.state_dict(), f"weights/{dataset_name}/netD_B.pth")


In [ ]:

gc.collect()
torch.cuda.empty_cache()
